In [1]:
import regex as re
import pandas as pd
from nltk.translate.bleu_score import SmoothingFunction
import random
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad")
validation = dataset['validation']

random.seed(42)

import pandas as pd
import regex as re
from tqdm import tqdm
import nltk
import RagNRollQA

c:\Users\zisak\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
len(validation['question'])

10570

In [3]:
# get length of questions which start with when and doesn't contain what, who, where, how and save the fake when questions indicies
when_questions = []
fake_when_questions = []
fake_when_questions_indices = []
for index, question in enumerate(validation['question']):
    if question.startswith("When") and not any(word in question.lower() for word in [", what", ", who", ", where", ", how"]):
        when_questions.append(question)
    elif question.startswith("When"):
        fake_when_questions.append(question)
        fake_when_questions_indices.append(index)
        
    
        
# remove rows from validation which contains questions inside fake_when_questions_indices
validation = validation.filter(lambda example, idx: idx not in fake_when_questions_indices, with_indices=True)
fake_when_questions_indices

[260,
 1420,
 5063,
 5788,
 6600,
 6808,
 6885,
 6904,
 7362,
 7459,
 8962,
 9035,
 9599,
 9667,
 9877,
 10253,
 10403,
 10422]

In [4]:
len(validation['question'])

10552

In [7]:

def QuestionStartsWith_Accuracy(dataset, startsWith, RagQA):

    correct = 0
    EM = 0
    BLEU = 0
    BLEU1 = 0
    BLEU2 = 0
    BLEU3 = 0
    BLEU4 = 0
    total = 0
    errors = []
    corrects = []
    empties = []
    kolo = 0
    for item in tqdm(dataset):
        random_number = random.randint(0, len(dataset))
        kolo += 1
        included_question = False
        try:
            context = item['context']
            context = re.sub(' +', ' ', context)
            
            question = item['question']
            tempQuestion = question.lower()
            question = re.sub(' +', ' ', question)
            
            answer = item['answers']['text'][0]
            title = item['title']
    
            # check if question starts with startsWith
            for start in startsWith:
                if tempQuestion.startswith(start):
                    included_question = True
                    break
            
            if included_question:
    
                total += 1
                if total == 74:
                    pass
                outputAnswer = RagQA.answer_question(question, context)
                if outputAnswer == "":
                    empties.append(kolo)
                    outputAnswer = "No_Answer_Found"
                
                # print("Question: " , question)
                # print("Answer: ", answer, "-------" , "Our Answer: ", outputAnswer)
                if outputAnswer in answer or answer in outputAnswer:
                    correct += 1
                    corrects.append(kolo)
                else:
                    # write to when_wrong_answers.txt
                    with open("results/where_wrong_answers.txt", "a") as file:
                        file.write(f"Question: {question}\n")
                        file.write(f"Answer: {answer}\n")
                        file.write(f"Our Answer: {outputAnswer}\n\n")
                        
                        
                if outputAnswer == answer:
                    EM += 1

                n = min(len(outputAnswer.split()), 4)
                if n == 0:
                    BLEUscore = 0
                else:
                    weights = [1.0/n]*n
                    smoothie = SmoothingFunction().method4
                    BLEUscore = nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=weights, smoothing_function=smoothie)
                    
                    BLEU += BLEUscore
                    BLEU1 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(1, 0, 0, 0), smoothing_function=smoothie)
                    BLEU2 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
                    BLEU3 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
                    BLEU4 += nltk.translate.bleu_score.sentence_bleu([answer], outputAnswer, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
                    

            if kolo % 1000 == 0:
                print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
                print(f"EM: {EM}, out of {total}: {100*EM/total}%")
                print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
                print(f"BLEU1: {BLEU1}, out of {total}: {100*BLEU1/total}%")
                print(f"BLEU2: {BLEU2}, out of {total}: {100*BLEU2/total}%")
                print(f"BLEU3: {BLEU3}, out of {total}: {100*BLEU3/total}%")
                print(f"BLEU4: {BLEU4}, out of {total}: {100*BLEU4/total}%")
        
        except Exception as e:
            print("title: ", title)
            print("Error in question number: ", total)
            print("Question: ", question)
            print("Answer: ", answer)
            print("Error: ", e)
            errors.append(total)
            # total -= 1
            print("\n\n")
    
    if total != 0:
        print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
        print(f"EM: {EM}, out of {total}: {100*EM/total}%")
        print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
        print(f"BLEU1: {BLEU1}, out of {total}: {100*BLEU1/total}%")
        print(f"BLEU2: {BLEU2}, out of {total}: {100*BLEU2/total}%")
        print(f"BLEU3: {BLEU3}, out of {total}: {100*BLEU3/total}%")
        print(f"BLEU4: {BLEU4}, out of {total}: {100*BLEU4/total}%")
    else:
        print("No Questions found with the given starting word")
    print("Errors: ", errors)
    print("Empties: ", len(empties), empties)
    return correct, total, errors, corrects


if __name__ == "__main__":

    startsWith = ["when "]
    RagQA = RagNRollQA.RagNRollQA()
    x = QuestionStartsWith_Accuracy(validation, startsWith, RagQA)

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  8%|▊         | 835/10552 [00:16<00:47, 205.91it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
  9%|▊         | 908/10552 [00:23<10:30, 15.30it/s] c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
  9%|▊         | 921/10552 [00:24<11:01, 14.57it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referre

Correct: 34, out of 51: 66.66666666666667%
EM: 22, out of 51: 43.13725490196079%
BLEU: 31.19980857987034, out of 51: 61.17609525464773%
BLEU1: 32.667965720809974, out of 51: 64.05483474668623%
BLEU2: 30.510495040110396, out of 51: 59.82450007864784%
BLEU3: 29.853393190260913, out of 51: 58.53606507894297%
BLEU4: 29.31288367915695, out of 51: 57.47624250815088%


 10%|▉         | 1042/10552 [00:35<16:05,  9.84it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 20%|█▉        | 2075/10552 [01:11<00:21, 395.71it/s]

Correct: 88, out of 129: 68.21705426356588%
EM: 59, out of 129: 45.736434108527135%
BLEU: 79.80435142786126, out of 129: 61.863838316171524%
BLEU1: 83.16735853261856, out of 129: 64.47082056792136%
BLEU2: 78.51499625966733, out of 129: 60.86433818578863%
BLEU3: 76.3987216977013, out of 129: 59.22381526953589%
BLEU4: 74.11135898374205, out of 129: 57.450665878869806%


 24%|██▎       | 2488/10552 [01:35<06:57, 19.34it/s] c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 28%|██▊       | 2992/10552 [02:00<08:10, 15.41it/s] 

...

Correct: 353, out of 551: 64.06533575317604%
EM: 225, out of 551: 40.83484573502722%
BLEU: 323.6112677413987, out of 551: 58.73162753927381%
BLEU1: 339.25762964418726, out of 551: 61.57125764867282%
BLEU2: 319.9741669648934, out of 551: 58.07153665424563%
BLEU3: 308.4896570672571, out of 551: 55.987233587523974%
BLEU4: 298.0155533927244, out of 551: 54.08630733080298%


 87%|████████▋ | 9150/10552 [05:29<00:21, 64.72it/s] c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 89%|████████▉ | 9370/10552 [05:36<01:30, 13.13it/s] c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 89%|████████▉ | 9374/10552 [05:36<01:30, 12.97it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(referring_head_lexeme)
 92%|█████████▏| 9660/10552 [05:51<00:27, 32.93it/s]c:\Users\zisak\anaconda3\Lib\site-packages\coreferee\tendencies.py:319: UserWarning: [W008] Evaluating Lexeme.similarity based on empty vectors.
  referred_head_lexeme.similarity(refer

Correct: 409, out of 648: 63.117283950617285%
EM: 256, out of 648: 39.50617283950617%
BLEU: 375.9565897800352, out of 648: 58.017992250005435%
BLEU1: 394.25360006648623, out of 648: 60.84160494853183%
BLEU2: 370.7638863254532, out of 648: 57.216649124298335%
BLEU3: 356.3985431510158, out of 648: 54.9997751776259%
BLEU4: 343.81492213234515, out of 648: 53.05785835375697%


100%|██████████| 10552/10552 [06:33<00:00, 26.82it/s]

Correct: 433, out of 677: 63.95864106351551%
EM: 273, out of 677: 40.32496307237814%
BLEU: 396.7466477836856, out of 677: 58.60364073614263%
BLEU1: 415.2567637898696, out of 677: 61.337778994072316%
BLEU2: 391.5667417064835, out of 677: 57.83851428456182%
BLEU3: 376.89789847480324, out of 677: 55.67177230056178%
BLEU4: 364.0704275655422, out of 677: 53.777020319873294%
Errors:  []
Empties:  24 [1624, 1763, 1951, 4632, 4636, 5022, 5045, 5679, 6498, 6583, 6849, 6955, 7372, 7595, 7720, 8253, 8894, 8895, 8905, 8927, 8932, 9205, 9512, 10089]


In [6]:
# [174, 315, 326, 349, 472, 488, 493, 498, 530, 667]
# when: 48%